# Data Migration Tracking

### Description
To track data migration status and filings done after migration

### Install Dependencies

In [1]:
# Run this in a cell if you haven't installed these packages
!pip install pandas openpyxl sqlalchemy numpy psycopg2-binary python-dotenv


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 1.2 MB/s eta 0:00:00a 0:00:01
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.5 MB/s eta 0:00:00
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 40.4 MB/s eta 0:00:0000:010:01
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 30.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 28.8 MB/s eta 0:00:0000:0100:01
Using cached python_dotenv-1.1.1-py3-none-any.whl (20 kB)
   ━━━━━━━━━━

### Define and get constants

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

# file path
GROUP_TABLE_FOLDER = os.getenv('GROUP_TABLE_FOLDER')
GROUP_TABLE_FILE_NAME = os.getenv('GROUP_TABLE_FILE_NAME')
OUTPUT_FOLDER = os.getenv('OUTPUT_FOLDER')

# corp_num column name
COLUMN_FOR_CORP_NUM = os.getenv('COLUMN_FOR_CORP_NUM')

# db configs
# Colin Extracts
COLIN_EXTRACT_DB = os.getenv('COLIN_EXTRACT_DB')
CE_HOST_URL = os.getenv('CE_HOST_URL')
CE_USERNAME = os.getenv('CE_USERNAME')
CE_PASSWORD = os.getenv('CE_PASSWORD')
CE_PORT = os.getenv('CE_PORT')
# Lear
LEAR_DB = os.getenv('LEAR_DB')
LEAR_HOST_URL = os.getenv('LEAR_HOST_URL')
LEAR_USERNAME = os.getenv('LEAR_USERNAME')
LEAR_PASSWORD = os.getenv('LEAR_PASSWORD')
LEAR_PORT = os.getenv('LEAR_PORT')

# Display stuff
PRINT_DIVIDER = "=" * 50

# Tracking Table Column Names
COLUMN_NAMES = {
    "corp_num": "Incorporation Number",
    "corp_name": "Company Name",
    "corp_type": "Type",
    "email": "Admin Email",
    "status": "Migration Status",
    "date": "Migrated Date",
    "filings": "Filings Done",
    "filing_date": "Last Filing Date"
}


In [3]:
# import necessary libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from typing import List, Any


In [8]:
# Helper function to establish database connection
def get_db_connection_string(
    host_address: str,
    database: str,
    user_name: str,
    db_password: str,
    port: str = "5432",
) -> str:
    """Create db connection string."""
    connection_string = (
        f"postgresql://{user_name}:{db_password}@{host_address}:{port}/{database}"
    )

    return connection_string


### Read Excel file and extract corp_nums and initialize the tracking dataframe

In [9]:
try:
    full_group_table_path = f"{GROUP_TABLE_FOLDER}/{GROUP_TABLE_FILE_NAME}"
    corp_nums_df = pd.read_excel(
        full_group_table_path, sheet_name="Sheet1", usecols=[COLUMN_FOR_CORP_NUM]
    )
    corp_nums_df = corp_nums_df.sort_values(COLUMN_FOR_CORP_NUM)

    corp_num_column_values = corp_nums_df[COLUMN_FOR_CORP_NUM].dropna().tolist()

    corp_nums_df = corp_nums_df.rename(
        columns={COLUMN_FOR_CORP_NUM: COLUMN_NAMES["corp_num"]}
    )
    print("Shape of data - Groups", corp_nums_df.shape)
    print("\nFirst 5 rows:")
    display(corp_nums_df.head())

    print(PRINT_DIVIDER)
    print(f"Found {len(corp_num_column_values)} corps in the group table")
    print(f"All corps:\n{corp_num_column_values}")
    print(PRINT_DIVIDER)

except Exception as e:
    print(f"Error reading Excel file: {e}")
    corp_num_column_values = []


Shape of data - Groups (64, 1)

First 5 rows:


,Incorporation Number
62,BC0754828
40,BC0769801
39,BC0910591
61,BC0934777
60,BC0934782


Found 64 corps in the group table
All corps:
['BC0754828', 'BC0769801', 'BC0910591', 'BC0934777', 'BC0934782', 'BC0971192', 'BC0988623', 'BC1033896', 'BC1034551', 'BC1041519', 'BC1049046', 'BC1055974', 'BC1065213', 'BC1072742', 'BC1080101', 'BC1082247', 'BC1090168', 'BC1105588', 'BC1113730', 'BC1139090', 'BC1140525', 'BC1161928', 'BC1169589', 'BC1172657', 'BC1173897', 'BC1179877', 'BC1180203', 'BC1185476', 'BC1196188', 'BC1201211', 'BC1208807', 'BC1211791', 'BC1221371', 'BC1228520', 'BC1245585', 'BC1246637', 'BC1249698', 'BC1250621', 'BC1256884', 'BC1292656', 'BC1297308', 'BC1302343', 'BC1308092', 'BC1341547', 'BC1341825', 'BC1344052', 'BC1361825', 'BC1363286', 'BC1387185', 'BC1395304', 'BC1400407', 'BC1411915', 'BC1414068', 'BC1417733', 'BC1428110', 'BC1440160', 'BC1475529', 'BC1481042', 'BC1483392', 'BC1484094', 'BC1484169', 'BC1484174', 'BC1507435', 'BC1507445']


### Get Data from Colin Extracts Database

In [10]:
# 1 - Connect to Colin Extracts DB
try:
    colin_extracts_connection_string = get_db_connection_string(
        CE_HOST_URL, COLIN_EXTRACT_DB, CE_USERNAME, CE_PASSWORD, CE_PORT
    )
    colin_extracts_engine = create_engine(colin_extracts_connection_string)

    # test connection
    with colin_extracts_engine.connect() as conn:
        conn.execute(text("SELECT 1"))
    print("Colin Extracts database connection successful")

except Exception as e:
    print(f"Connection to Colin Extracts failed: {e}")
    colin_extracts_engine = None

print(PRINT_DIVIDER)


Colin Extracts database connection successful


In [11]:
# 2 - Get corp names
if colin_extracts_engine and corp_num_column_values:
    try:
        values_str = "', '".join(str(val) for val in corp_num_column_values)
        in_clause = f"corp_num IN ('{values_str}')"

        query = f"""
        SELECT corp_num, corp_name
        FROM public.corp_name
        WHERE {in_clause}
        AND corp_name_typ_cd IN ('CO', 'NB')
        AND end_event_id IS NULL
        ORDER BY corp_num
        """
        corp_names_data = pd.read_sql(query, colin_extracts_engine)
        print(f"Found {len(corp_names_data)} matches.")
        print(PRINT_DIVIDER)

        add_name_df = corp_nums_df.copy(deep=True)
        add_name_df = add_name_df.merge(
            corp_names_data,
            left_on=COLUMN_NAMES["corp_num"],
            right_on="corp_num",
            how="left",
        )
        add_name_df = add_name_df.drop("corp_num", axis=1)
        add_name_df = add_name_df.rename(
            columns={"corp_name": COLUMN_NAMES["corp_name"]}
        )

        print(f"Total {len(add_name_df)} rows.")
        with pd.option_context("display.max_rows", None):
            display(add_name_df)
    except Exception as e:
        print(f"Failed to execute queries: {e}")


Found 64 matches.
Total 64 rows.


,Incorporation Number,Company Name
0,BC0754828,CLINSCAPE CONSULTING INC.
1,BC0769801,FREE THE GOLDFISH COACHING INC.
2,BC0910591,MOKO PROPERTY GROUP INC.
3,BC0934777,WILSON 5 FOUNDATION MANAGEMENT LTD.
4,BC0934782,LOW TIDE PROPERTIES TRUSTEE LTD.
5,BC0971192,BOWERY FUNDING ULC
6,BC0988623,NHI DENIS PERSONAL REAL ESTATE CORPORATION
7,BC1033896,1033896 B.C. LTD.
8,BC1034551,CHRISTIAN P. GAUTHIER LAW CORPORATION
9,BC1041519,FRIND ENTERPRISES LTD.


In [12]:
# 3 - Get Corp types and admin_email from corporation table
if colin_extracts_engine and corp_num_column_values:
    try:
        values_str = "', '".join(str(val) for val in corp_num_column_values)
        in_clause = f"corp_num IN ('{values_str}')"

        type_email_query = f"""
        SELECT corp_num, corp_type_cd, admin_email
        FROM public.corporation
        WHERE {in_clause}
        ORDER BY corp_num
        """
        type_email_data = pd.read_sql(type_email_query, colin_extracts_engine)
        print(f"Found {len(type_email_data)} matches.")
        print(PRINT_DIVIDER)

        add_type_email_df = add_name_df.copy(deep=True)
        add_type_email_df = add_type_email_df.merge(
            type_email_data,
            left_on=COLUMN_NAMES["corp_num"],
            right_on="corp_num",
            how="left",
        )
        add_type_email_df = add_type_email_df.drop("corp_num", axis=1)
        add_type_email_df = add_type_email_df.rename(
            columns={
                "corp_type_cd": COLUMN_NAMES["corp_type"],
                "admin_email": COLUMN_NAMES["email"],
            }
        )

        print(f"Total {len(add_type_email_df)} rows.")
        with pd.option_context("display.max_rows", None):
            display(add_type_email_df)
    except Exception as e:
        print(f"Failed to execute queries: {e}")


Found 64 matches.
Total 64 rows.


,Incorporation Number,Company Name,Type,Admin Email
0,BC0754828,CLINSCAPE CONSULTING INC.,BC,vancorp@bennettjones.com
1,BC0769801,FREE THE GOLDFISH COACHING INC.,BC,vannotices@mcmillan.ca
2,BC0910591,MOKO PROPERTY GROUP INC.,BC,vannotices@mcmillan.ca
3,BC0934777,WILSON 5 FOUNDATION MANAGEMENT LTD.,BC,VanCorp@bennettjones.com
4,BC0934782,LOW TIDE PROPERTIES TRUSTEE LTD.,BC,VanCorp@bennettjones.com
5,BC0971192,BOWERY FUNDING ULC,ULC,vannotices@mcmillan.ca
6,BC0988623,NHI DENIS PERSONAL REAL ESTATE CORPORATION,BC,vannotices@mcmillan.ca
7,BC1033896,1033896 B.C. LTD.,BC,vancorp@bennettjones.com
8,BC1034551,CHRISTIAN P. GAUTHIER LAW CORPORATION,BC,VanCorp@bennettjones.com
9,BC1041519,FRIND ENTERPRISES LTD.,BC,vannotices@mcmillan.ca


In [13]:
# 4 - Get status and migrated date from corp_processing table
if colin_extracts_engine and corp_num_column_values:
    try:
        values_str = "', '".join(str(val) for val in corp_num_column_values)
        in_clause = f"corp_num IN ('{values_str}')"

        migration_query = f"""
        SELECT corp_num, processed_status, create_date
        FROM public.corp_processing
        WHERE {in_clause}
        ORDER BY corp_num
        """
        migration_data = pd.read_sql(migration_query, colin_extracts_engine)
        print(f"Found {len(migration_data)} matches.")
        print(PRINT_DIVIDER)

        migration_df = add_type_email_df.copy(deep=True)
        migration_df = migration_df.merge(
            migration_data,
            left_on=COLUMN_NAMES["corp_num"],
            right_on="corp_num",
            how="left",
        )
        migration_df = migration_df.drop("corp_num", axis=1)
        migration_df = migration_df.rename(
            columns={
                "processed_status": COLUMN_NAMES['status'],
                "create_date": COLUMN_NAMES['date'],
            }
        )
        migration_df[COLUMN_NAMES['status']] = np.where(migration_df[COLUMN_NAMES['status']] == 'COMPLETED', 'Migrated', 'Pending')
        migration_df[COLUMN_NAMES['date']] = pd.to_datetime(migration_df[COLUMN_NAMES['date']].dt.date)

        print(f"Total {len(migration_df)} rows.")
        with pd.option_context("display.max_rows", None):
            display(migration_df)
        print(PRINT_DIVIDER)
    except Exception as e:
        print(f"Failed to execute queries: {e}")


Found 64 matches.
Total 64 rows.


,Incorporation Number,Company Name,Type,Admin Email,Migration Status,Migrated Date
0,BC0754828,CLINSCAPE CONSULTING INC.,BC,vancorp@bennettjones.com,Migrated,2025-04-16
1,BC0769801,FREE THE GOLDFISH COACHING INC.,BC,vannotices@mcmillan.ca,Migrated,2025-06-09
2,BC0910591,MOKO PROPERTY GROUP INC.,BC,vannotices@mcmillan.ca,Migrated,2025-06-09
3,BC0934777,WILSON 5 FOUNDATION MANAGEMENT LTD.,BC,VanCorp@bennettjones.com,Migrated,2025-04-16
4,BC0934782,LOW TIDE PROPERTIES TRUSTEE LTD.,BC,VanCorp@bennettjones.com,Migrated,2025-04-16
5,BC0971192,BOWERY FUNDING ULC,ULC,vannotices@mcmillan.ca,Migrated,2025-06-09
6,BC0988623,NHI DENIS PERSONAL REAL ESTATE CORPORATION,BC,vannotices@mcmillan.ca,Migrated,2025-06-09
7,BC1033896,1033896 B.C. LTD.,BC,vancorp@bennettjones.com,Migrated,2025-06-13
8,BC1034551,CHRISTIAN P. GAUTHIER LAW CORPORATION,BC,VanCorp@bennettjones.com,Migrated,2025-06-13
9,BC1041519,FRIND ENTERPRISES LTD.,BC,vannotices@mcmillan.ca,Migrated,2025-06-09


### Get Data from LEAR DB

In [14]:
# Connect to LEAR DB
try:
    lear_connection_string = get_db_connection_string(
        LEAR_HOST_URL, LEAR_DB, LEAR_USERNAME, LEAR_PASSWORD, LEAR_PORT
    )
    lear_engine = create_engine(lear_connection_string)

    # test connection
    with lear_engine.connect() as lear_conn:
        lear_conn.execute(text("SELECT 1"))
    print("LEAR database connection successful")

except Exception as e:
    print(f"Connection to LEAR failed: {e}")
    lear_engine = None

print(PRINT_DIVIDER)


LEAR database connection successful


In [15]:
# Functions to get LEAR filings and the latest filing date

def get_business_ids(db_connection: Any, corp_nums_list: List) -> pd.DataFrame:
    """
    Query the Businesses table to get business IDs for given corp_nums_list
    """
    if not corp_nums_list:
        print("Empty corp_nums_list")
        print(PRINT_DIVIDER)
        return pd.DataFrame()
    
    values_str = "', '".join(str(val) for val in corp_nums_list)
    in_clause = f"identifier IN ('{values_str}')"

    query = f"""
    SELECT identifier, id as business_id
    FROM public.businesses
    WHERE {in_clause}
    """

    df = pd.read_sql_query(query, db_connection)
    print(f"Found {len(df)} businesses matching the corp_nums/identifiers")
    print(PRINT_DIVIDER)
    return df


def get_lear_filings(db_connection: Any, business_ids_df: pd.DataFrame) -> pd.DataFrame:
    """
    Query Filings table for business_ids and filter for Source = 'LEAR'
    """
    try:
        if business_ids_df.empty:
            print("Empty business_ids_df")
            print(PRINT_DIVIDER)
            return pd.DataFrame()
        
        business_id_list = business_ids_df['business_id'].tolist()
        value_holders = ','.join(str(val) for val in business_id_list)

        query = f"""
        SELECT business_id, filing_date, filing_type, status, source
        FROM public.filings
        WHERE business_id IN ({value_holders})
        AND source = 'LEAR'
        ORDER BY business_id, filing_date DESC
        """

        df = pd.read_sql_query(query, db_connection)
        print(f"Found {len(df)} filings with source  'LEAR'")
        print(PRINT_DIVIDER)
        return df
    except Exception as e:
        print(f"Error getting LEAR filings: {e}")
        print(PRINT_DIVIDER)
        return pd.DataFrame()


def process_found_lear_filings(filings_df: pd.DataFrame, business_ids_df: pd.DataFrame) -> pd.DataFrame:
    """
    Group filing types with status by business_id and get the latest filing date
    Returns DataFrame with identifiers, lear_filings_found, last_filing_date
    """
    if filings_df.empty:
        return pd.DataFrame(columns=['identifiers', 'lear_filings_found', 'last_filing_date'])
    
    filings_df['formatted_filing_type'] = filings_df['filing_type'].apply(camel_to_title_case)
    # Create filing type with status string
    filings_df['filing_with_status'] = filings_df['formatted_filing_type'] + ' (' + filings_df['status'] + ')'

    # Group by business_id and aggregate
    aggregated = filings_df.groupby('business_id').agg({
        'filing_with_status': lambda x: ', '.join(sorted(set(x))),  # Unique filing types with status joined by comma
        'filing_date': 'max'  # Latest filing date
    }).reset_index()


    aggregated.columns = ['business_id', 'lear_filings_found', 'last_filing_date']

    result_df = business_ids_df.merge(aggregated, on='business_id', how='inner')
    result_df['last_filing_date'] = pd.to_datetime(result_df['last_filing_date'].dt.date)
    
    result_df = result_df.rename(columns={'identifier': COLUMN_NAMES['corp_num']})
    result_df = result_df[[COLUMN_NAMES['corp_num'], 'lear_filings_found', 'last_filing_date']]

    print(f"Processed {len(result_df)} businesses with LEAR filings")
    print(PRINT_DIVIDER)
    return result_df


def camel_to_title_case(camel_str):
    """Convert camelCase to Title Case (e.g., 'annualReport' -> 'Annual Report')"""
    import re
    # Insert space before uppercase letters that follow lowercase letters
    result = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', camel_str)
    # Capitalize first letter of each word
    return result.title()
    

In [16]:
# Put LEAR filings and the latest filing date into the main dataframe
try:
    business_ids_df = get_business_ids(lear_engine, corp_num_column_values)
    lear_filings_df = get_lear_filings(lear_engine, business_ids_df)
    lear_results_df = process_found_lear_filings(lear_filings_df, business_ids_df)
    with pd.option_context("display.max_rows", None):
        display(lear_results_df)
    print(PRINT_DIVIDER)

    current_df = migration_df.copy(deep=True)
    current_df = current_df.merge(
        lear_results_df,
        on=COLUMN_NAMES['corp_num'],
        how='left'
    )
    current_df['lear_filings_found'] = current_df['lear_filings_found'].fillna('')
    current_df['last_filing_date'] = current_df['last_filing_date'].astype(str).replace('NaT', '')

    current_df = current_df.rename(
        columns={
            'lear_filings_found': COLUMN_NAMES['filings'],
            'last_filing_date': COLUMN_NAMES['filing_date']
        }
    )

    with pd.option_context("display.max_rows", None):
        display(current_df)
except Exception as e:
    print(f"Error putting LEAR filings data into the main dataframe:\n {e}")
    print(PRINT_DIVIDER)


Found 64 businesses matching the corp_nums/identifiers
Found 6 filings with source  'LEAR'
Processed 3 businesses with LEAR filings


,Incorporation Number,lear_filings_found,last_filing_date
0,BC1179877,Annual Report (COMPLETED),2025-06-24
1,BC1484174,"Annual Report (COMPLETED), Change Of Directors...",2025-06-24
2,BC1507445,Agm Location Change (COMPLETED),2025-06-24


,Incorporation Number,Company Name,Type,Admin Email,Migration Status,Migrated Date,Filings Done,Last Filing Date
0,BC0754828,CLINSCAPE CONSULTING INC.,BC,vancorp@bennettjones.com,Migrated,2025-04-16,,
1,BC0769801,FREE THE GOLDFISH COACHING INC.,BC,vannotices@mcmillan.ca,Migrated,2025-06-09,,
2,BC0910591,MOKO PROPERTY GROUP INC.,BC,vannotices@mcmillan.ca,Migrated,2025-06-09,,
3,BC0934777,WILSON 5 FOUNDATION MANAGEMENT LTD.,BC,VanCorp@bennettjones.com,Migrated,2025-04-16,,
4,BC0934782,LOW TIDE PROPERTIES TRUSTEE LTD.,BC,VanCorp@bennettjones.com,Migrated,2025-04-16,,
5,BC0971192,BOWERY FUNDING ULC,ULC,vannotices@mcmillan.ca,Migrated,2025-06-09,,
6,BC0988623,NHI DENIS PERSONAL REAL ESTATE CORPORATION,BC,vannotices@mcmillan.ca,Migrated,2025-06-09,,
7,BC1033896,1033896 B.C. LTD.,BC,vancorp@bennettjones.com,Migrated,2025-06-13,,
8,BC1034551,CHRISTIAN P. GAUTHIER LAW CORPORATION,BC,VanCorp@bennettjones.com,Migrated,2025-06-13,,
9,BC1041519,FRIND ENTERPRISES LTD.,BC,vannotices@mcmillan.ca,Migrated,2025-06-09,,


### Save the final dataframe to an Excel file

In [17]:
from openpyxl.styles import Font, Alignment

def format_and_save_excel(df: pd.DataFrame, file_save_path: str = 'output.xlsx', sheet_name: str = 'Sheet1') -> None:
    """Save to Excel file with basic freeze and alignment."""
    
    # Generate unique filename with date and incremental number
    file_save_path = generate_unique_filename(file_save_path)

    with pd.ExcelWriter(file_save_path, engine='openpyxl') as writer:
        # write the dataframe
        df.to_excel(writer, sheet_name=sheet_name, index=False)

        # get the workbook and worksheet
        worksheet = writer.sheets[sheet_name]

        # Freeze the 1st row
        worksheet.freeze_panes = 'A2'

        # Make header bold
        for cell in worksheet[1]:  # First row (header)
            cell.font = Font(bold=True)
            cell.alignment = Alignment(horizontal='left')
    
        # Left align all other cells
        for row in worksheet.iter_rows(min_row=2):  # Skip header row
            for cell in row:
                cell.alignment = Alignment(horizontal='left')
        
        # Adjust column widths
        for column in worksheet.columns:
            max_length = 0
            column_letter = column[0].column_letter

            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = max_length + 2
            worksheet.column_dimensions[column_letter].width = adjusted_width
        
        print(f"DataFrame saved to {file_save_path} with frozen header and left alignment")


import os
from datetime import datetime

def generate_unique_filename(original_path: str) -> str:
    """
    Generate unique filename with date and incremental number if file exists.
    Example: 'output.xlsx' -> 'output_20250626.xlsx' -> 'output_20250626_02.xlsx'
    """
    # Get directory, filename, and extension
    directory = os.path.dirname(original_path)
    filename = os.path.basename(original_path)
    name, ext = os.path.splitext(filename)

    # Add today's date
    today = datetime.now().strftime('%Y%m%d')
    new_filename = f"{name}_{today}{ext}"
    new_path = os.path.join(directory, new_filename)

    # If file doesn't exist, return the new path
    if not os.path.exists(new_path):
        return new_path
    
    # If file exists, add incremental number
    counter = 2
    while True:
        incremental_filename = f"{name}_{today}_{counter:02d}{ext}"
        incremental_path = os.path.join(directory, incremental_filename)
        
        if not os.path.exists(incremental_path):
            return incremental_path
        
        counter += 1
        
        # Safety check to prevent infinite loop
        if counter > 999:
            raise Exception("Too many files with the same name pattern")



In [18]:
# Export formatted Excel file
current_df[COLUMN_NAMES['date']] = current_df[COLUMN_NAMES['date']].astype(str)
output_path = f"{OUTPUT_FOLDER}/test_result.xlsx"
format_and_save_excel(current_df, output_path)


DataFrame saved to /mnt/c/Users/EASPAN/Downloads/test_result_20250626.xlsx with frozen header and left alignment
